<a href="https://colab.research.google.com/github/smthomas1704/restoration-rag/blob/main/search_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Testing testing

!git clone https://github.com/smthomas1704/restoration-rag.git

fatal: destination path 'restoration-rag' already exists and is not an empty directory.


In [26]:
!pip install -r restoration-rag/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [28]:
!pip3 freeze > restoration-rag/requirements.txt

In [34]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_PATH = 'restoration-rag/data' #Your root data folder path
DB_FAISS_PATH = 'vectorstore/db_faiss'

loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()

print(len(documents))
print(documents[0].page_content[0:100])


38
1194  |     J Appl Ecol. 2023;60:1194–1205.
 wileyonlinelibrary.com/journal/jpeReceived: 1 April 202


In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
splits = text_splitter.split_documents(documents)
print(len(splits), splits[0])

518 page_content='1194\u2003 |\u2003    J Appl Ecol. 2023;60:1194–1205.\n wileyonlinelibrary.com/journal/jpeReceived: 1 April 2021\u2003 |\u2003Accepted: 4 March 2023\nDOI: 10.1111/1365-2664.14395  \nRESEARCH ARTICLE\nPrioritizing areas for ecological restoration: A participatory \napproach based on cost-  effectiveness\nElysa Silva1\u2003|   Walid  Naji1\u2003|   Pietro Salvaneschi2\u2003|   Emilio  Climent-  Gil1\u2003|   \nMchich  Derak3\u2003|   Germán López1\u2003|   Andreu Bonet1\u2003|   Antonio Aledo1\u2003|   \nJordi  Cortina-  Segarra1' metadata={'source': 'restoration-rag/data/Journal of Applied Ecology - 2023 - Silva - Prioritizing areas for ecological restoration  A participatory approach based.pdf', 'page': 0}


In [36]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [37]:
db = FAISS.from_documents(splits, embeddings)
db.save_local(DB_FAISS_PATH)

In [39]:
!curl localhost:8080/generate -X POST -H 'Content-Type: application/json' -d '{"inputs": "Tips for reforestation", "parameters": { "max_new_tokens":64}}' #Replace the locahost with the IP visible to the machine running the notebook


In [40]:
import langchain
from queue import Queue
from typing import Any
from langchain.llms.huggingface_text_gen_inference import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import LLMResult
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from anyio.from_thread import start_blocking_portal #For model callback streaming

langchain.debug=True

#vector db path
DB_FAISS_PATH = 'vectorstore/db_faiss'

#Llama2 TGI models host port
LLAMA2_7B_HOSTPORT = "http://localhost:8080/" #Replace the locahost with the IP visible to the machine running the notebook
LLAMA2_13B_HOSTPORT = "http://localhost:8080/" #Add your own host ports for model switching. You can host another TGI model on same instance on a different port.


model_dict = {
    "7b-chat" : LLAMA2_7B_HOSTPORT,
    "13b-chat" : LLAMA2_13B_HOSTPORT,
}

system_message = {"role": "system", "content": "You are a helpful assistant."}